In [52]:
from bin.game_interfaces_binding import *

In [53]:
action_path = "test-cases/match-364.txt"
map_path = "test-cases/map-364-ctu-test-game-1.txt"

In [54]:
from utils import load_map

mapdata = load_map(map_path)
mapdata

{'game_map': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [55]:
def map_tile_to_bitmask(tile):
    if tile == 1:
        return 1 << TileMask.POND.value
    elif tile == 2:
        return 1 << TileMask.CASTLE.value
    return 0

map_formatted = [[map_tile_to_bitmask(x) for x in row] for row in mapdata['game_map']]

map_formatted

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [56]:
counter = 0
craftsman_strid_to_intid = {}
craftsmen_formatted = []
for c in mapdata['craftsmen']['team1']:
    new_craftsman = Craftsman(counter, c['x'], c['y'], True)
    craftsmen_formatted.append(new_craftsman)
    craftsman_strid_to_intid[c['id']] = counter
    counter += 1
for c in mapdata['craftsmen']['team2']:
    new_craftsman = Craftsman(counter, c['x'], c['y'], False)
    craftsmen_formatted.append(new_craftsman)
    craftsman_strid_to_intid[c['id']] = counter
    counter += 1

print(craftsman_strid_to_intid)

{'78817d36': 0, '47bca97a': 1, 'f00948bd': 2, '23245594': 3, '049f27e0': 4, 'ab62f118': 5, 'e842024f': 6, 'fec58a12': 7, '385da14a': 8, 'be8880f5': 9, '1e5ce1c3': 10, '453da780': 11, '8d550b3d': 12, '7e8b2c9d': 13, '44666481': 14, '7e7fdfe8': 15, '15d07528': 16, '711ec844': 17, '3d55a647': 18, '9d5d5222': 19, '25cb9690': 20, '28d86b0a': 21, '05d65e61': 22, '47e7757e': 23, '685d15c8': 24, '3953920a': 25, '5bd0411e': 26, 'cc190b85': 27, '0c84c3ac': 28, '1e33d021': 29, '30415de9': 30, 'b71d0679': 31, '7e9bd2bb': 32, '90cb86cb': 33, '25e7773c': 34, '107428e0': 35, '4281ce71': 36, 'f9e33007': 37, 'df06ae00': 38, '3ee1ae43': 39, '09c2d283': 40, '09f4cae6': 41, '985f430a': 42, '97af8cf6': 43, 'dc4c123d': 44, '63208621': 45, 'e7eda736': 46, '4685cb94': 47, 'bfbdd0d7': 48, '6a6b662c': 49}


In [57]:
actiontxt = open(action_path, "r").read()

# Split the input string based on "- -"
turns = actiontxt.strip().split("- -")


# Remove empty strings from the list
turns = [section.strip() for section in turns]
print (turns)

# Convert each section to an array of arrays
actions = [[action.split(' ') for action in (turn.split('\n') if turn else [])] for turn in turns]

idxToActionEnum = [
    [ActionType.MOVE, SubActionType.MOVE_UP],
    [ActionType.MOVE, SubActionType.MOVE_DOWN],
    [ActionType.MOVE, SubActionType.MOVE_LEFT],
    [ActionType.MOVE, SubActionType.MOVE_RIGHT],
    [ActionType.MOVE, SubActionType.MOVE_UP_LEFT],
    [ActionType.MOVE, SubActionType.MOVE_UP_RIGHT],
    [ActionType.MOVE, SubActionType.MOVE_DOWN_LEFT],
    [ActionType.MOVE, SubActionType.MOVE_DOWN_RIGHT],
    [ActionType.BUILD, SubActionType.BUILD_UP],
    [ActionType.BUILD, SubActionType.BUILD_DOWN],
    [ActionType.BUILD, SubActionType.BUILD_LEFT],
    [ActionType.BUILD, SubActionType.BUILD_RIGHT],
    [ActionType.DESTROY, SubActionType.DESTROY_UP],
    [ActionType.DESTROY, SubActionType.DESTROY_DOWN],
    [ActionType.DESTROY, SubActionType.DESTROY_LEFT],
    [ActionType.DESTROY, SubActionType.DESTROY_RIGHT],
    [ActionType.STAY, SubActionType.STAY],
  ]

actions = [[GameAction(craftsman_strid_to_intid[action[0]], *idxToActionEnum[int(action[1])]) for action in turn] for turn in actions]

['78817d36 10\n47bca97a 15\nf00948bd 16\n23245594 15\n049f27e0 15\nab62f118 10\ne842024f 10\nfec58a12 4\n385da14a 16\nbe8880f5 16\n1e5ce1c3 14\n453da780 13\n8d550b3d 3\n7e8b2c9d 16\n44666481 16\n7e7fdfe8 10\n15d07528 14\n711ec844 2\n3d55a647 16\n9d5d5222 0\n25cb9690 10\n28d86b0a 6\n05d65e61 14\n47e7757e 16\n685d15c8 0', '3953920a 8\n5bd0411e 16\ncc190b85 15\n0c84c3ac 10\n1e33d021 1\n30415de9 2\nb71d0679 15\n7e9bd2bb 15\n90cb86cb 15\n25e7773c 7\n107428e0 10\n4281ce71 5\nf9e33007 13\ndf06ae00 16\n3ee1ae43 16\n09c2d283 16\n09f4cae6 16\n985f430a 16\n97af8cf6 13\ndc4c123d 16\n63208621 12\ne7eda736 10\n4685cb94 16\nbfbdd0d7 9\n6a6b662c 7', '78817d36 14\n47bca97a 14\nf00948bd 9\n23245594 13\n049f27e0 6\nab62f118 13\ne842024f 16\nfec58a12 14\n385da14a 12\nbe8880f5 13\n1e5ce1c3 14\n453da780 14\n8d550b3d 9\n7e8b2c9d 4\n44666481 16\n7e7fdfe8 8\n15d07528 15\n711ec844 12\n3d55a647 2\n9d5d5222 1\n25cb9690 16\n28d86b0a 4\n05d65e61 0\n47e7757e 8\n685d15c8 9', '3953920a 14\n5bd0411e 10\ncc190b85 11\n0c

In [58]:
go = GameOptions()
go.mapWidth = mapdata['game_settings']['map_width']
go.mapHeight = mapdata['game_settings']['map_height']
go.maxTurns =  mapdata['game_settings']['max_turn']
go.wallCoeff =  mapdata['score_coefficients']['wall']
go.castleCoeff =   mapdata['score_coefficients']['castle']
go.territoryCoeff =   mapdata['score_coefficients']['territory']

game = Game(
    go,
    map_formatted,
    craftsmen_formatted
)

In [59]:
for i, turn in enumerate(actions):
    for action in turn:
        game.addAction(action)
    game.nextTurn()
    print(f'Turn {i+2}:',game.getCurrentState().map.calcPoints(game.gameOptions, True), game.getCurrentState().map.calcPoints(game.gameOptions, False))
    print(game.getCurrentState().map.printMap())
    print()

print(game.getCurrentState().map.calcPoints(game.gameOptions, True), game.getCurrentState().map.calcPoints(game.gameOptions, False))

Turn 2: 5 0
0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000100000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000010000 0000010000 0000000000 0000000000 0000000000 0000010000 0000000000 0000000000 0000010000 0000000000 0000000000 0000000000 
0000000000 0000010000 0000100000 0000000000 0000000000 0000100000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000010000 0000000001 0000000000 0000000000 0000000000 0000000000 
0000000000 0000000100 0000010000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 000000

In [60]:
import threading
import time

In [61]:
start = time.time()

def run_game():
    for i in range(100):
        go = GameOptions()
        go.mapWidth = mapdata['game_settings']['map_width']
        go.mapHeight = mapdata['game_settings']['map_height']
        go.maxTurns =  mapdata['game_settings']['max_turn']
        go.wallCoeff =  mapdata['score_coefficients']['wall']
        go.castleCoeff =   mapdata['score_coefficients']['castle']
        go.territoryCoeff =   mapdata['score_coefficients']['territory']

        game = Game(
            go,
            map_formatted,
            craftsmen_formatted
        )

        for turn in actions:
            for action in turn:
                game.addAction(action)
            game.nextTurn()
    # print(time.time() - start)

threads = []
for i in range(24):
    t = threading.Thread(target=run_game)
    t.start()
    threads.append(t)

for t in threads:
    t.join()
print(time.time() - start)

18.590600728988647
